In [95]:
import os
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Unified data directories
Datadir = r'C:\Users\Administrator\Documents\MnewData'
output_path = r'C:\Users\Administrator\Documents\MnewData'

def list_csv_files(directory, keyword):
    """List CSV files in the specified directory that contain the keyword, case-insensitively."""
    return [file for file in os.listdir(directory) if keyword.lower() in file.lower() and file.lower().endswith('.csv')]

def read_and_process_files(directory, keyword, columns=None, encoding='gbk'):
    """Read and process CSV files, returning a concatenated DataFrame."""
    files = list_csv_files(directory, keyword)
    dfs = []
    for file in files:
        file_path = os.path.join(directory, file)
        try:
            df = pd.read_csv(file_path, encoding=encoding, usecols=columns,na_values=["n/a", "na", "-"]) if columns else pd.read_csv(file_path, encoding=encoding,skiprows=2, header=0, na_values=["n/a", "na", "-"])
            df.columns = df.columns.str.replace(' ', '')
            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file}: {e}")
    return pd.concat(dfs, ignore_index=True).drop_duplicates() if dfs else pd.DataFrame()




# Antenna Planning    df_AP

In [96]:
# 读取数据并重命名列
df_AP = read_and_process_files(Datadir, '天线安装规划', ['网元标识', '远端射频单元编号', '本地小区标识1'])

# 重命名列
df_AP.rename(columns={
    '远端射频单元编号': '射频单元编号',
    '本地小区标识1': '小区本地ID'
}, inplace=True)

In [97]:
df_AP.head(10)

,网元标识,射频单元编号,小区本地ID
0,6355679,1,2
4,6355679,0,1
8,6355947,3,4
9,6355947,1,2
16,6355947,0,1
20,6355947,2,3
24,6355826,1,5
28,6355826,4,2
32,6355826,0,4
36,6355826,3,1


# Radio Frequency Planning  df_RF

In [98]:

df_RF = read_and_process_files(Datadir, '射频单元规划', ['网元标识', '射频单元编号', '射频单元RRU安装经度', '射频单元RRU安装纬度'])
# 重命名列
df_RF.rename(columns={
    '射频单元RRU安装经度': 'Longitude',
    '射频单元RRU安装纬度': 'Latitude'
}, inplace=True)


In [99]:
df_RF.head(10)

,网元标识,射频单元编号,Longitude,Latitude
0,6355688,2,114.259369,29.931835
1,6355688,0,114.259369,29.931835
2,6355688,1,114.259369,29.931835
3,6337869,1,110.546023,32.064548
4,6337869,2,110.546023,32.064548
5,6337869,0,110.546023,32.064548
6,6355693,2,114.087013,29.536091
7,6355693,1,114.087013,29.536091
8,6355693,0,114.087013,29.536091
9,6334748,1,110.080330,33.169849


# Cell parameters df_CP

In [100]:
df_CP = read_and_process_files(Datadir, 'NR小区', ['网元标识', '网元名称', '小区本地ID', '小区友好名', 'Nr小区工作频段'])
df_CP.rename(columns={
    '网元名称': 'BBU机房',
    '小区友好名': '小区名称',
    'Nr小区工作频段': '工作频段'
}, inplace=True)
df_CP['工作频段'] = df_CP['工作频段'].str.split('(').str[0]
df_CP['基站名称'] = df_CP['小区名称'].str.replace(r'(-26.*|-07.*)', '', regex=True)

In [101]:
df_CP.head(10)

,网元标识,BBU机房,小区本地ID,小区名称,工作频段,基站名称
0,6355688,咸安胡翰林700M-D5H,2,咸安胡翰林700M-D5H-0712,band28,咸安胡翰林700M-D5H
1,6355688,咸安胡翰林700M-D5H,3,咸安胡翰林700M-D5H-0713,band28,咸安胡翰林700M-D5H
2,6355688,咸安胡翰林700M-D5H,1,咸安胡翰林700M-D5H-0711,band28,咸安胡翰林700M-D5H
3,6337869,房县化龙镇700M-D5H,1,房县化龙镇700M-D5H-0711,band28,房县化龙镇700M-D5H
4,6337869,房县化龙镇700M-D5H,2,房县化龙镇700M-D5H-0712,band28,房县化龙镇700M-D5H
5,6337869,房县化龙镇700M-D5H,3,房县化龙镇700M-D5H-0713,band28,房县化龙镇700M-D5H
6,6355693,咸安黄铁700M-D5H,1,咸安黄铁700M-D5H-0711,band28,咸安黄铁700M-D5H
7,6355693,咸安黄铁700M-D5H,3,咸安黄铁700M-D5H-0713,band28,咸安黄铁700M-D5H
8,6355693,咸安黄铁700M-D5H,2,咸安黄铁700M-D5H-0712,band28,咸安黄铁700M-D5H
9,6334748,郧西二天门700M-D5H,1,郧西二天门700M-D5H-0711,band28,郧西二天门700M-D5H


# Read Map

In [102]:
def read_hubei_map_files(directory, filename):
    """Read the Hubei map file."""
    file_path = os.path.join(directory, filename)
    return gpd.read_file(file_path)

In [103]:
gdffile = '湖北省村级边界.geojson'
gdf = read_hubei_map_files(Datadir, gdffile)

In [104]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd

def process_rf_ap_cp(df_ap, df_rf, df_cp, gdf):
    """
    合并天线规划数据、射频单元规划数据和小区规划数据，并与地理数据进行空间连接。

    参数:
    df_ap (pd.DataFrame): 天线规划数据。
    df_rf (pd.DataFrame): 射频单元规划数据。
    df_cp (pd.DataFrame): 小区规划数据。
    gdf (gpd.GeoDataFrame): 地理数据。

    返回值:
    gpd.GeoDataFrame: 合并后的GeoDataFrame，包含规整后的列名。
    """
    # 合并天线规划数据和射频单元规划数据
    df_rfap = pd.merge(df_ap, df_rf, how='inner', 
                       left_on=['网元标识', '射频单元编号'],
                       right_on=['网元标识', '射频单元编号'],
                       suffixes=('_df_AP', '_df_RF'))

    # 选择并重命名列
    df_rfap = df_rfap[['网元标识', '小区本地ID', '射频单元编号', 'Longitude', 'Latitude']]

    # 检查并删除重复数据
    df_rfap = df_rfap.drop_duplicates()

    # 合并射频单元规划数据和小区规划数据
    df_rac = pd.merge(df_rfap, df_cp, how='inner', 
                      left_on=['网元标识', '小区本地ID'],
                      right_on=['网元标识', '小区本地ID'],
                      suffixes=('_df_RFAP', '_df_CP'))

    # 选择并重命名列
    df_rac = df_rac[['网元标识', '小区本地ID', 'BBU机房', '基站名称', '小区名称', '工作频段', 'Longitude', 'Latitude']]

    # 检查并删除重复数据
    df_rac = df_rac.drop_duplicates()

    # 创建几何图形列表（点）
    geometry = [Point(xy) for xy in zip(df_rac['Longitude'], df_rac['Latitude'])]

    # 将 pandas DataFrame 转换为 GeoDataFrame
    df_rac = gpd.GeoDataFrame(df_rac, geometry=geometry)

    # 设置 CRS
    if df_rac.crs is None:
        df_rac = df_rac.set_crs(gdf.crs)

    # 空间连接
    gdf_rac = gpd.sjoin(df_rac, gdf, how='inner', predicate='within')

    # 确保 '网元标识' 和 '小区本地ID' 列都是字符串类型
    gdf_rac['网元标识'] = gdf_rac['网元标识'].astype(str)
    gdf_rac['小区本地ID'] = gdf_rac['小区本地ID'].astype(str)

    # 增加列ID
    gdf_rac['ID'] = gdf_rac['网元标识'] + '_' + gdf_rac['小区本地ID']

    # 选择并重命名列
    gdf_rac = gdf_rac[['ID', '网元标识', '小区本地ID', 'BBU机房', '基站名称', '小区名称', '工作频段', 'Longitude', 'Latitude', 'SJGZQYMC', 'DSJGZQYMC', 'QXJGZQYMC', 'XZJGZQYMC', 'CJGZQYMC']].rename(
        columns={'SJGZQYMC': '省份', 'DSJGZQYMC': '地市', 'QXJGZQYMC': '县区', 'XZJGZQYMC': '镇区', 'CJGZQYMC': '村区'}
    )

    # 检查并删除 GeoDataFrame 中的重复数据
    gdf_rac = gdf_rac.drop_duplicates()

    return gdf_rac

# 使用示例
gdf_RAC = process_rf_ap_cp(df_AP, df_RF, df_CP, gdf)

In [105]:
gdf_RAC.head(10)

,ID,网元标识,小区本地ID,BBU机房,基站名称,小区名称,工作频段,Longitude,Latitude,省份,地市,县区,镇区,村区
0,6355679_2,6355679,2,通山富池700M-D5H,通山富池700M-D5H,通山富池700M-D5H-0712,band28,114.291809,29.575943,湖北省,咸宁市,崇阳县,路口镇,泉口村
1,6355679_1,6355679,1,通山富池700M-D5H,通山富池700M-D5H,通山富池700M-D5H-0711,band28,114.291809,29.575943,湖北省,咸宁市,崇阳县,路口镇,泉口村
2,6355947_4,6355947,4,通山车田村700M-D5H,通山洪港村五组700M-D5H,通山洪港村五组700M-D5H-0711,band28,114.845510,29.507170,湖北省,咸宁市,通山县,洪港镇,毛田河村
3,6355947_2,6355947,2,通山车田村700M-D5H,通山车田村700M-D5H,通山车田村700M-D5H-0712,band28,114.828488,29.500281,湖北省,咸宁市,通山县,洪港镇,车田村
4,6355947_1,6355947,1,通山车田村700M-D5H,通山车田村700M-D5H,通山车田村700M-D5H-0711,band28,114.828488,29.500281,湖北省,咸宁市,通山县,洪港镇,车田村
5,6355947_3,6355947,3,通山车田村700M-D5H,通山车田村700M-D5H,通山车田村700M-D5H-0713,band28,114.828488,29.500281,湖北省,咸宁市,通山县,洪港镇,车田村
6,6355826_5,6355826,5,崇阳叶门700M-D5H,崇阳叶门700M-D5H,崇阳叶门700M-D5H-0712,band28,113.905539,29.490531,湖北省,咸宁市,崇阳县,石城镇,方山村
7,6355826_2,6355826,2,崇阳叶门700M-D5H,崇阳荻洲700M-D5H,崇阳荻洲700M-D5H-0712,band28,113.963381,29.488554,湖北省,咸宁市,崇阳县,石城镇,荻洲村
8,6355826_4,6355826,4,崇阳叶门700M-D5H,崇阳叶门700M-D5H,崇阳叶门700M-D5H-0711,band28,113.905539,29.490531,湖北省,咸宁市,崇阳县,石城镇,方山村
9,6355826_1,6355826,1,崇阳叶门700M-D5H,崇阳荻洲700M-D5H,崇阳荻洲700M-D5H-0711,band28,113.963381,29.488554,湖北省,咸宁市,崇阳县,石城镇,荻洲村


# Read KPI file and Regular format  df_KPI

In [106]:
df_kpi = read_and_process_files(Datadir, 'DT_PowerBI指标通报计数器_')

def process_df_kpi(df):
    # 提取 NB 和 nrCellCfg 并创建 ID 列
    df[['NB', 'nrCellCfg']] = df['对象'].str.extract(r'gNB=(\d+),nrCellCfg=(\d+)')
    df['ID'] = df['NB'] + '_' + df['nrCellCfg']
    
    # 重新排列列顺序
    new_order = ['ID', 'NB', 'nrCellCfg'] + [col for col in df.columns if col not in ['ID', 'NB', 'nrCellCfg']]
    df = df[new_order].copy()  # 使用 .copy() 创建一个新的 DataFrame 副本

    # 提取括号中的内容作为新的列名
    df.columns = [re.search(r'\((.*?)\)', col).group(1) if re.search(r'\((.*?)\)', col) else col for col in df.columns]

    # 删除指定的列
    columns_to_drop = ['对象', 'Nr小区工作频段', 'MHz', '逻辑小区id']
    df.drop(columns=columns_to_drop, inplace=True)

    # 转换开始时间和结束时间为日期格式
    df.loc[:, '开始时间'] = pd.to_datetime(df['开始时间']).dt.date
    df.loc[:, '结束时间'] = pd.to_datetime(df['结束时间']).dt.date

    # 将 R 开头的列转换为整数类型
    r_columns = [col for col in df.columns if col.startswith('R')]
    df.loc[:, r_columns] = df[r_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    # 将 K 开头的列转换为带两位小数的浮点类型
    k_columns = [col for col in df.columns if col.startswith('K')]
    df.loc[:, k_columns] = df[k_columns].apply(pd.to_numeric, errors='coerce').fillna(0).round(2)

    # 检查并删除重复数据
    df = df.drop_duplicates()

    return df

df_KPI = process_df_kpi(df_kpi)


In [107]:
df_KPI.head(10)

,ID,NB,nrCellCfg,开始时间,结束时间,R2035_003,R2035_013,R2035_026,R2007_001,R2007_002,...,R1001_001,R1001_004,R1001_007,R1001_008,R1001_012,R1001_015,R1001_018,R1001_019,K1009_001,K1009_002
0,6299318_1,6299318,1,2024-10-01,2024-10-02,12.0,12.0,232.0,21071.0,21043.0,...,47654.0,10985.0,318.0,0.0,47409.0,10966.0,315.0,0.0,18.52,1.07
1,6299318_1,6299318,1,2024-10-02,2024-10-03,20.0,20.0,583.0,62210.0,62081.0,...,96563.0,21972.0,674.0,0.0,95554.0,21915.0,673.0,0.0,29.83,1.88
2,6299318_1,6299318,1,2024-10-03,2024-10-04,30.0,29.0,709.0,73361.0,73156.0,...,111052.0,26896.0,755.0,0.0,110232.0,26834.0,746.0,0.0,39.98,2.34
3,6299318_1,6299318,1,2024-10-04,2024-10-05,20.0,19.0,536.0,56781.0,56673.0,...,92325.0,21638.0,606.0,0.0,91646.0,21585.0,603.0,0.0,29.06,1.62
4,6299318_1,6299318,1,2024-10-05,2024-10-06,13.0,13.0,397.0,44852.0,44738.0,...,74425.0,17539.0,500.0,0.0,73916.0,17493.0,499.0,0.0,20.91,1.42
5,6299318_1,6299318,1,2024-10-06,2024-10-07,6.0,6.0,200.0,21420.0,21394.0,...,37589.0,9302.0,377.0,0.0,37362.0,9283.0,377.0,0.0,11.81,0.93
6,6299318_2,6299318,2,2024-10-01,2024-10-02,21.0,21.0,460.0,21807.0,21679.0,...,74621.0,15028.0,560.0,0.0,74221.0,14999.0,558.0,0.0,30.38,1.42
7,6299318_2,6299318,2,2024-10-02,2024-10-03,27.0,27.0,590.0,65412.0,65159.0,...,103271.0,20207.0,830.0,0.0,102220.0,20157.0,824.0,0.0,33.85,1.79
8,6299318_2,6299318,2,2024-10-03,2024-10-04,38.0,38.0,610.0,77036.0,76713.0,...,117944.0,22895.0,892.0,0.0,116880.0,22827.0,884.0,0.0,32.78,2.21
9,6299318_2,6299318,2,2024-10-04,2024-10-05,29.0,26.0,551.0,59900.0,59714.0,...,102291.0,20786.0,756.0,0.0,101446.0,20730.0,755.0,0.0,28.51,1.45


In [108]:
# 查看 df_KPI['NB'] 的数据类型
nb_dtype = df_KPI['NB'].dtype
print("数据类型为:", nb_dtype)

数据类型为: object


# read BBU&RRU Power  df_BRP

In [109]:
df_bup = read_and_process_files(Datadir, 'DT_BBU功耗_')

# 从 '对象' 字段中提取 'BBU名称' 和 'NB'
df_bup['BBU名称'] = df_bup['对象'].str.extract(r'^(.*?)(?=\(gNB=)')
df_bup['NB'] = df_bup['对象'].str.extract(r'gNB=(\d+)').astype(str)

# 根据 BBU名称 确定 站型
df_bup['站型'] = df_bup['BBU名称'].apply(lambda x: '宏站' if 'D5H' in x else ('微站' if 'D5M' in x else ('室分' if 'D5S' in x else '未知')))

# 将指定列转换为整数
int_columns = ['gNB基站CPU平均负荷(R1056_001)[%]', 'gNB基站CPU峰值负荷(R1056_002)[%]']
df_bup[int_columns] = df_bup[int_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# 将 '开始时间' 转换为日期格式
df_bup['开始时间'] = pd.to_datetime(df_bup['开始时间']).dt.date

# 将指定列转换为浮点数并保留四位小数
float_columns = ['BBU功耗[千瓦时]', 'BBU功耗(R1054_001)[W]']
df_bup[float_columns] = df_bup[float_columns].apply(lambda x: pd.to_numeric(x, errors='coerce').fillna(0).round(4))

# 按指定列表提取数据
columns_to_select = ['BBU名称', 'NB', '站型', '开始时间', 'BBU功耗[千瓦时]', 
                     'gNB基站CPU平均负荷(R1056_001)[%]', 'gNB基站CPU峰值负荷(R1056_002)[%]', 'BBU功耗(R1054_001)[W]']
df_BUP = df_bup[columns_to_select]



In [110]:
# 查看 df_KPI['NB'] 的数据类型
nb_dtype = df_BUP['NB'].dtype
print("数据类型为:", nb_dtype)

数据类型为: object


In [111]:
df_rup = read_and_process_files(Datadir, 'DT_RRU功耗_')

# 使用正则表达式提取 BBUID 和 RRUID
df_rup['NB'] = df_rup['对象'].str.extract(r'gNB=(\d+)').astype(str)
df_rup['RRUID'] = df_rup['对象'].str.extract(r'invRRU=(\d+)').astype(str)

# 将 'AAU功耗[千瓦时]' 列转换为浮点数并保留四位小数
df_rup['AAU功耗[千瓦时]'] = pd.to_numeric(df_rup['AAU功耗[千瓦时]'], errors='coerce').round(4).fillna(0)

# 将 '开始时间' 列转换为日期格式
df_rup['开始时间'] = pd.to_datetime(df_rup['开始时间']).dt.date
# 按指定列表提取数据
columns_to_select = ['NB', 'RRUID', '开始时间', 'AAU功耗[千瓦时]']
df_RUP = df_rup[columns_to_select]


In [112]:
df_RUP.sample(10)

,NB,RRUID,开始时间,AAU功耗[千瓦时]
948490,6337209,5,2024-12-01,9.1603
14531,6313218,11,2024-10-06,1.3730
778873,6337100,3,2024-10-20,3.7123
598807,6299361,2,2024-11-17,1.5608
598611,6299352,1,2024-11-17,2.0130
870695,6311683,3,2024-11-06,3.4318
593744,6355810,3,2024-12-06,1.8778
46527,6346367,0,2024-10-04,0.0000
283996,6326391,2,2024-11-03,0.3793
397401,6299439,3,2024-11-22,1.8323


In [113]:
def calculate_antenna_and_power(bbu_df, rru_df, kpi_df):

    """
    在BBU DataFrame中计算天线数量、RRU总功耗、数据业务流量和VoNR语音话务量，并添加频段。

    参数:
    bbu_df (pd.DataFrame): BBU数据。
    rru_df (pd.DataFrame): RRU数据。
    kpi_df (pd.DataFrame): KPI数据。

    返回值:
    pd.DataFrame: 更新后的BBU DataFrame，包含天线数量、RRU总功耗、数据业务流量、VoNR语音话务量和频段。
    """
    
    # 计算天线数量
    antenna_count = rru_df.groupby(['NB', '开始时间'])['RRUID'].nunique().reset_index(name='天线数量')
    
    # 计算RRU总功耗
    total_rru_power = rru_df.groupby(['NB', '开始时间'])['AAU功耗[千瓦时]'].sum().reset_index(name='RRU总功耗')
    
    # 合并天线数量和RRU总功耗到BBU DataFrame
    bbu_df = pd.merge(bbu_df, antenna_count, on=['NB', '开始时间'], how='left')
    bbu_df = pd.merge(bbu_df, total_rru_power, on=['NB', '开始时间'], how='left')
    
    # 计算频段
    bbu_df['频段'] = bbu_df['BBU名称'].apply(lambda x: '700M' if '700M' in x else '2.6G')
    
    # 计算数据业务流量和VoNR语音话务量，并保留两位小数
    kpi_df["数据业务流量"] = (kpi_df["R1012_001"] + kpi_df["R1012_002"]).round(2)
    kpi_df["VoNR语音话务量"] = (kpi_df["K1009_001"] / 4).round(2)
    
    # 按照 ['NB', '开始时间'] 分组并求和
    traffic_data = kpi_df.groupby(['NB', '开始时间'])[['数据业务流量', 'VoNR语音话务量']].sum().reset_index()
    
    # 将“数据业务流量”转换为百万单位并保留两位小数
    traffic_data['数据业务流量'] = (traffic_data['数据业务流量'] / 1000000).round(2)
    
    # 合并数据业务流量和VoNR语音话务量到BBU DataFrame
    bbu_df = pd.merge(bbu_df, traffic_data, on=['NB', '开始时间'], how='left')
    
    # 四舍五入功耗数据
    bbu_df['BBU功耗[千瓦时]'] = bbu_df['BBU功耗[千瓦时]'].round(4)
    bbu_df['RRU总功耗'] = bbu_df['RRU总功耗'].round(4)
    bbu_df['总功耗'] = (bbu_df['BBU功耗[千瓦时]'] + bbu_df['RRU总功耗']).round(4)

        # 检查并删除重复数据
    bbu_df = bbu_df.drop_duplicates()

    return bbu_df

# 调用函数
df_BRP = calculate_antenna_and_power(df_BUP, df_RUP, df_KPI)


In [114]:
df_BRP.head(10)

,BBU名称,NB,站型,开始时间,BBU功耗[千瓦时],gNB基站CPU平均负荷(R1056_001)[%],gNB基站CPU峰值负荷(R1056_002)[%],BBU功耗(R1054_001)[W],天线数量,RRU总功耗,频段,数据业务流量,VoNR语音话务量,总功耗
0,丹江蔡湾700M-D5H,6299318,宏站,2024-10-01,6.7090,3,8,26836.0,3.0,8.0398,700M,134.65,13.11,14.7488
1,丹江蔡湾700M-D5H,6299318,宏站,2024-10-02,6.7693,4,10,27077.0,3.0,12.3043,700M,257.48,16.89,19.0736
2,丹江蔡湾700M-D5H,6299318,宏站,2024-10-03,6.7758,5,11,27103.0,3.0,12.5263,700M,254.36,18.94,19.3021
3,丹江蔡湾700M-D5H,6299318,宏站,2024-10-04,6.7503,4,9,27001.0,3.0,10.4333,700M,193.25,15.21,17.1836
4,丹江蔡湾700M-D5H,6299318,宏站,2024-10-05,6.7398,4,9,26959.0,3.0,9.4111,700M,166.15,12.75,16.1509
5,丹江蔡湾700M-D5H,6299318,宏站,2024-10-06,6.7263,3,7,26905.0,3.0,7.3561,700M,116.52,10.19,14.0824
6,丹江凤凰山700M-D5H,6299319,宏站,2024-10-01,0.0000,2,6,0.0,3.0,5.3991,700M,62.13,8.68,5.3991
7,丹江凤凰山700M-D5H,6299319,宏站,2024-10-02,0.0000,2,6,0.0,3.0,6.5009,700M,122.18,17.50,6.5009
8,丹江凤凰山700M-D5H,6299319,宏站,2024-10-03,0.0000,2,6,0.0,3.0,6.2923,700M,111.59,9.28,6.2923
9,丹江凤凰山700M-D5H,6299319,宏站,2024-10-04,0.0000,2,6,0.0,3.0,5.9468,700M,91.43,11.06,5.9468


In [115]:
df_BRP.head()

,BBU名称,NB,站型,开始时间,BBU功耗[千瓦时],gNB基站CPU平均负荷(R1056_001)[%],gNB基站CPU峰值负荷(R1056_002)[%],BBU功耗(R1054_001)[W],天线数量,RRU总功耗,频段,数据业务流量,VoNR语音话务量,总功耗
0,丹江蔡湾700M-D5H,6299318,宏站,2024-10-01,6.7090,3,8,26836.0,3.0,8.0398,700M,134.65,13.11,14.7488
1,丹江蔡湾700M-D5H,6299318,宏站,2024-10-02,6.7693,4,10,27077.0,3.0,12.3043,700M,257.48,16.89,19.0736
2,丹江蔡湾700M-D5H,6299318,宏站,2024-10-03,6.7758,5,11,27103.0,3.0,12.5263,700M,254.36,18.94,19.3021
3,丹江蔡湾700M-D5H,6299318,宏站,2024-10-04,6.7503,4,9,27001.0,3.0,10.4333,700M,193.25,15.21,17.1836
4,丹江蔡湾700M-D5H,6299318,宏站,2024-10-05,6.7398,4,9,26959.0,3.0,9.4111,700M,166.15,12.75,16.1509


In [116]:
df_BRP.sample(10)

,BBU名称,NB,站型,开始时间,BBU功耗[千瓦时],gNB基站CPU平均负荷(R1056_001)[%],gNB基站CPU峰值负荷(R1056_002)[%],BBU功耗(R1054_001)[W],天线数量,RRU总功耗,频段,数据业务流量,VoNR语音话务量,总功耗
210698,丹江邮政楼-D5H,6337268,宏站,2024-10-29,22.4880,9,16,89952.0,6.0,75.1486,2.6G,1419.60,173.25,97.6366
202063,丹江丁家营镇轻工服装产业园区-D5H,6334918,宏站,2024-10-25,5.8113,8,9,23245.0,2.0,4.4788,2.6G,17.49,3.19,10.2901
97673,丹江天星庙700M-D5H,6337789,宏站,2024-11-10,6.4428,9,10,25771.0,6.0,9.1039,700M,60.39,4.17,15.5467
108921,通山南岭口700M-D5H,6312789,宏站,2024-11-16,4.2995,5,13,17198.0,3.0,3.8126,700M,9.46,2.86,8.1121
205595,竹山桥东村-D5H,6337615,宏站,2024-10-22,6.3840,10,11,25536.0,3.0,20.4988,2.6G,138.26,37.90,26.8828
4905,麻城大田铺700M-D5H,6326342,宏站,2024-10-04,4.6330,6,10,18532.0,3.0,4.4643,700M,71.75,5.38,9.0973
7575,郧西唐家坪700M-D5H,6335372,宏站,2024-10-04,4.2355,5,13,16942.0,3.0,3.4611,700M,15.05,1.95,7.6966
39643,红安姜牌700M-D5H,6326383,宏站,2024-10-20,6.5500,2,3,26200.0,3.0,4.4105,700M,44.62,7.51,10.9605
52841,竹山鲁家坝700M-D5H,6299490,宏站,2024-10-23,6.2988,2,6,25195.0,3.0,4.3636,700M,12.97,6.71,10.6624
101502,郧西店子七里扁700M-D5H,6341983,宏站,2024-11-10,4.2163,5,13,16865.0,3.0,3.4826,700M,17.02,7.51,7.6989


# Save File

In [117]:
def save_dataframe_to_csv(df, filename):
    """Save DataFrame to CSV file."""
    df.to_csv(os.path.join(output_path, filename), index=False, encoding='utf-8-sig')
    print(f"DataFrame exported to {filename}")

In [118]:
# Save results
save_dataframe_to_csv(gdf_RAC, 'gdf_RAC.csv')
save_dataframe_to_csv(df_KPI, 'df_KPI.csv')
save_dataframe_to_csv(df_BRP, 'df_BRP.csv')

DataFrame exported to gdf_RAC.csv
DataFrame exported to df_KPI.csv
DataFrame exported to df_BRP.csv
